In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import random_split, Dataset, DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np
import random
import pickle
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
print("Device:", device)

Device: cpu


In [ ]:
with open('/content/drive/MyDrive/train.pkl', 'rb') as f:
    data_set = pickle.load(f)

In [ ]:
data_set[0][1]

0

In [ ]:
data = []
targets = []
for i in data_set:
  data.append(i[0])
  targets.append(i[1])

In [ ]:
print(data[1000])
print(targets)

[ 67.  12.  12. 159.  12.  28.  28.  12.  13.  12.  28.  12.  28.  28.
  28.  28.  12.  29. 125.  28.  28.  12. 159.  12.  15.  13.  79.  28.
  78.  28.  28.  12.  28.  28.  47.   0.]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
from collections import Counter

labels = [label for _, label in data_set]

class_counts = Counter(labels)
classes = []
print("Liczba klas:", len(class_counts))
for class_label, count in class_counts.items():
    classes.append(count)
classes = np.array(classes)
print(classes)

Liczba klas: 5
[1630  478  154  441  236]


In [ ]:
class VariableLenDataset(Dataset):
    def __init__(self, in_data, target):
        self.data = [(x, y) for x, y in zip(in_data, target)]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        in_data, target = self.data[idx]
        return in_data, target

In [ ]:
train_indices = np.random.rand(len(data_set))>0.3

In [ ]:
train_indices

In [ ]:
data_train = []
targets_train = []
data_test = []
targets_test = []
for i in range(len(data_set)):
  if train_indices[i] == True:
    data_train.append(data_set[i][0])
    targets_train.append(data_set[i][1])
  else:
    data_test.append(data_set[i][0])
    targets_test.append(data_set[i][1])

In [ ]:
# dataset_size = len(data_set)

# train_size = int(0.8 * dataset_size)
# test_size = dataset_size - train_size


# train_set = VariableLenDataset(data[:train_indices], targets[:train_indices])
# test_set = VariableLenDataset(data[train_indices:], targets[train_indices:])

In [ ]:
train_set = VariableLenDataset(data_train, targets_train)
test_set = VariableLenDataset(data_test, targets_test)

In [ ]:
print(train_set[2])

In [ ]:
pad = -100

# def pad_collate(batch, pad_value=pad):
#     xx, yy = zip(*batch)
#     xx = [torch.tensor(x) for x in xx]
#     x_lens = [len(x) for x in xx]
#     yy_pad = torch.tensor(yy)
#     y_lens = [1] * len(yy)
#     # print(xx)
#     # print(yy)

#     xx_pad = pad_sequence(xx, batch_first=True, padding_value=pad_value)
#     # yy_pad = pad_sequence(yy, batch_first=True, padding_value=pad_value)

#     return xx_pad, yy_pad, x_lens, y_lens

def pad_collate(batch, pad_value=pad):
    try:
        xx, yy = zip(*batch)
    except ValueError as e:
        print(f"Error while unpacking batch: {e}")
        print(f"Batch content: {batch}")
        raise

    xx = [torch.tensor(x) for x in xx]
    x_lens = [len(x) for x in xx]
    yy_pad = torch.tensor(yy)
    y_lens = [1] * len(yy)

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=pad_value)

    return xx_pad, yy_pad, x_lens, y_lens

In [ ]:
batch_size=64

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=pad_collate)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, drop_last=False, collate_fn=pad_collate)

In [ ]:
# dataset_size = len(data)

# train_size = int(0.8 * dataset_size)
# test_size = dataset_size - train_size

# train_set, test_set = random_split(data, [train_size, test_size])

# batch_size=4
# trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, collate_fn=pad_collate)
# testloader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, drop_last=False, collate_fn=pad_collate)

In [ ]:
data_set[0]


In [ ]:
print(len(train_set[0][0]))
print(len(train_set[1][0]))

In [ ]:
next(iter(train_loader))

(tensor([[  -1.,   -1.,  144.,  ..., -100., -100., -100.],
         [   0.,   92.,   47.,  ..., -100., -100., -100.],
         [  -1.,   -1.,    0.,  ...,  119.,  119.,   12.],
         ...,
         [   0.,   47.,  180.,  ..., -100., -100., -100.],
         [   0.,    0.,   82.,  ..., -100., -100., -100.],
         [  -1.,   -1.,    0.,  ..., -100., -100., -100.]], dtype=torch.float64),
 tensor([1, 0, 1, 2, 0, 1, 0, 1, 0, 1, 1, 3, 0, 0, 3, 1, 0, 0, 0, 1, 1, 0, 0, 0,
         1, 0, 4, 1, 3, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 3, 2, 0, 0, 4, 0, 3, 1, 1,
         0, 4, 0, 0, 1, 3, 0, 0, 0, 1, 0, 0, 1, 0, 4, 4]),
 [2237,
  100,
  4082,
  228,
  564,
  72,
  172,
  128,
  368,
  2322,
  718,
  232,
  222,
  80,
  414,
  240,
  60,
  692,
  80,
  800,
  189,
  92,
  556,
  48,
  1083,
  330,
  189,
  1576,
  260,
  176,
  378,
  2070,
  111,
  452,
  195,
  256,
  348,
  358,
  52,
  240,
  210,
  136,
  57,
  198,
  48,
  222,
  72,
  284,
  868,
  236,
  256,
  507,
  588,
  216,
  688,
  48,
 

In [ ]:
class LSTMRegressor(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, out_size, bidirectional = False):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        if bidirectional:
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional=bidirectional, dropout=0.4)
        self.fc = nn.Linear(320, out_size)
        # self.fc = nn.Linear(hidden_size*90*self.bidirectional, out_size)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        return hidden, state

    def forward(self, x, hidden):
        # x = torch.transpose(x,0,1)
        x = x.float()
        all_outputs, hidden = self.lstm(x, hidden)
        # all_outputs = torch.transpose(all_outputs,0,1)
        # out = torch.flatten(all_outputs,1)
        # output, _ = pad_packed_sequence(all_outputs, batch_first=True)
        # output = output[:, -1, :]

        h_n, _ = hidden
        h_n = torch.flatten(h_n, 1)  # Flatten the hidden state
        # print(h_n[0])
        x = self.fc(h_n)

        # hidden = torch.flatten(hidden,1)
        # print(hidden[0])
        # x = self.fc(hidden)
        return x, hidden

model = LSTMRegressor(1,5,2,5).to(device)
model

LSTMRegressor(
  (lstm): LSTM(1, 5, num_layers=2, dropout=0.4)
  (fc): Linear(in_features=320, out_features=5, bias=True)
)

In [ ]:
class LSTMRegressor(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, out_size, bidirectional = False):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        if bidirectional:
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional=bidirectional, dropout=0.4)
        self.fc = nn.Linear(hidden_size * self.bidirectional, out_size)


    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        return hidden, state

    def forward(self, x, hidden):
        x = x.float()
        all_outputs, hidden = self.lstm(x, hidden)
        h_n, _ = hidden
        # h_n = torch.flatten(h_n, 1)
        # print(h_n)
        x = self.fc(h_n)
        return x, hidden

model = LSTMRegressor(1,5,2,5).to(device)
model

LSTMRegressor(
  (lstm): LSTM(1, 5, num_layers=2, dropout=0.4)
  (fc): Linear(in_features=5, out_features=5, bias=True)
)

In [ ]:
with torch.no_grad():
    for x, targets, x_len, target_len in test_loader:
        x = x.to(device).unsqueeze(2)
        targets = targets.to(device)
        hidden, state = model.init_hidden(x.shape[0])
        hidden, state = hidden.to(device), state.to(device)

        # print(targets)

#         x = torch.transpose(x, 0, 1)
#         preds, _ = model(x, (hidden, state))
#         preds = torch.transpose(preds, 0, 1)

        x_packed = pack_padded_sequence(x, x_len, batch_first=True, enforce_sorted=False)
        # print(x_packed)
        preds_packed, _ = model(x_packed, (hidden, state))
        preds, _ = model(x_packed, (hidden, state))
        # preds, pred_len = pad_packed_sequence(preds_packed, batch_first=True, padding_value=pad)


        # print(preds)
        print("Preds shape before squeeze:", preds.shape)
        preds = preds.squeeze(0)
        # preds = preds.view(-1, 5)  # Zamienia wymiary [1, 64, 5] na [64, 5]
        preds = preds[-1]
        mask_tgt = targets != pad
        # print(targets)
        # print(preds[0])
        print("Targets shape:", targets.shape)
        print("Preds shape:", preds.shape)
        print("Mask shape:", mask_tgt.shape)
        print(torch.abs(preds[mask_tgt] - targets[mask_tgt]).mean())
        print()

Preds shape before squeeze: torch.Size([2, 64, 5])
Targets shape: torch.Size([64])
Preds shape: torch.Size([64, 5])
Mask shape: torch.Size([64])


RuntimeError: The size of tensor a (5) must match the size of tensor b (64) at non-singleton dimension 1

In [ ]:
with torch.no_grad():
    hidden, state = model.init_hidden(len(test_data))
    hidden, state = hidden.to(device), state.to(device)
    preds,_ =  model(test_data.to(device).unsqueeze(2), (hidden,state))
print(f"Accuracy: {(torch.argmax(preds,1).cpu()==test_targets).sum().item()/len(test_targets):.3}")